In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import copy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
from torch.nn.utils import parameters_to_vector, vector_to_parameters

In [2]:
# Graph implementation
def generate_graph(cluster_sizes=[100,100], pin=0.5, pout=0.01, seed=0):
    """Generate a random connected graph"""
    probs = np.array([[pin, pout],[pout, pin]])
    while True:
        g = nx.stochastic_block_model(cluster_sizes, probs)
        if nx.algorithms.components.is_connected(g):
            return g


cluster_sizes = [10, 10]
pin = 0.5
#pout = 0.01
pout = 0.01
seed = 0
alpha = 1e-2
lamda = 1e-3
eta = 1e-2
d0 = 9
no_users = sum(cluster_sizes)
batch_size = 50
epochs = 1
it = 2000
G = generate_graph(cluster_sizes, pin, pout, seed)

#nx.draw(G, with_labels=True, node_size=100, alpha=1, linewidths=10)
#plt.show()

In [3]:
# Metropolis weights 
number_nodes = G.number_of_nodes()
weights = np.zeros([number_nodes, number_nodes])
for edge in G.edges():
  i, j = edge[0], edge[1]
  weights[i - 1][j - 1] = 1 / (1 + np.max([G.degree(i), G.degree(j)]))
  weights[j - 1][i - 1] = weights[i - 1][j - 1]

print(weights)

weights = weights + np.diag(1 - np.sum(weights, axis=0))

metropolis_weights = weights
print(metropolis_weights)


[[0.         0.         0.         0.125      0.         0.14285714
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.14285714
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.16666667]
 [0.         0.         0.         0.125      0.14285714 0.14285714
  0.14285714 0.14285714 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.14285714]
 [0.125      0.         0.125      0.         0.125      0.125
  0.125      0.125      0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.125     ]
 [0.         0.         0.14285714 0.125      0.         0.14285714
  0.14285714 0.         0.16666667 0.         0.         0.
  0.         0.         0

In [4]:
def load_dataset():
    transforms_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,),(0.3081,))])
    mnist_data_train = datasets.MNIST('./data/mnist', train=True, download=True, transform=transforms_mnist)
    mnist_data_test = datasets.MNIST('./data/mnist', train=False, download=True, transform=transforms_mnist)

    return mnist_data_train, mnist_data_test

In [5]:
def degrees(A):
    """Return the degrees of each node of a graph from its adjacency matrix"""
    return np.sum(A, axis=0).reshape(A.shape[0], 1)

def node_degree(n, G):
    cnt = 0
    for i in G.neighbors(n):
        cnt += 1
    return cnt

def get_neighbors(n, G):
    neighbors_list = []
    for i in G.neighbors(n):
        neighbors_list.append(int(i))
    return neighbors_list

In [6]:
# Dataset partitioning
def random_split(X, y, n, seed):
    """Equally split data between n agents"""
    rng = np.random.default_rng(seed)
    perm = rng.permutation(y.size)
    X_split = np.array_split(X[perm], n)  #np.stack to keep as a np array
    y_split = np.array_split(y[perm], n)
    return X_split, y_split





X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')


X, y = random_split(X_train, y_train, no_users, 1234)

In [7]:
X_train.shape

(14087, 9)

In [8]:
datapoints = {}
count = 0
W1 = np.array([2.0, 2.0, 3.0, 3.0])
W2 = np.array([-2.0, 2.0, 3.0, -3.0])
W3 = 2 * W1
W4 = 2  * W2
W = [W1, W2]
m = 200
n = 4

scaler = [1.0, -1.0]

noise_sd = 0.001
for i, cluster_size in enumerate(cluster_sizes):
    for j in range(cluster_size):
        features = np.random.normal(loc=0.0, scale=1.0, size=(m, n))
        label = np.dot(features, W[i ]) + np.random.normal(0,noise_sd)
        data = X[count]
        data[:, 0:4] *= scaler[i]
        datapoints[count] = {
                'features': data,
                'degree': node_degree(count, G),
                'label': y[count],
                'neighbors': get_neighbors(count, G),
                'exact_weights': torch.from_numpy(W[i])
            }
        count += 1

In [9]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = torch.FloatTensor(data)
        self.targets = torch.FloatTensor(targets).unsqueeze(-1)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y
    
    def __len__(self):
        return len(self.data)


In [10]:
class MLP_Net(nn.Module):
    def __init__(self, user_id):
        super(MLP_Net, self).__init__()
        self.fc1 = nn.Linear(9, 4, bias=False)
        self.fc2 = nn.Linear(4, 1, bias=False)
        #self.fc3 = nn.Linear(200, 10)
        self.user_id = user_id

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        output = self.fc2(x)
        #output = self.fc3(x)
        return output

In [11]:
from typing import Iterable, Optional

def grads_to_vector(parameters: Iterable[torch.Tensor]) -> torch.Tensor:
    r"""Convert parameters to one vector

    Args:
        parameters (Iterable[Tensor]): an iterator of Tensors that are the
            parameters of a model.

    Returns:
        The parameters represented by a single vector
    """
    # Flag for the device where the parameter is located
    param_device = None

    vec = []
    for param in parameters:
        # Ensure the parameters are located in the same device
        param_device = param.grad

        vec.append(param_device.view(-1))
    return torch.cat(vec)

In [12]:
model = MLP_Net(user_id=0)

lr = 0.01

dataloader = DataLoader(MyDataset(datapoints[19]["features"], datapoints[19]["label"]), batch_size=100, shuffle=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
for i in range(100):
    for (x, y) in dataloader:
        criterion = nn.MSELoss()
        optimizer.zero_grad()
        yhat = model(x)
        print(y.size())
        print(yhat.size())
        loss = criterion(yhat, y)
        
        loss.backward()
        print(i, loss, grads_to_vector(model.parameters()), parameters_to_vector(model.parameters()))
        #optimizer.step()
        new_model = parameters_to_vector(model.parameters()) - lr * grads_to_vector(model.parameters())
        vector_to_parameters(parameters=model.parameters(), vec=new_model)
        #if i % 50 ==0:
            #lr *= 0.9
            

#parameters_to_vector(model.parameters())

torch.Size([100, 1])
torch.Size([100, 1])
0 tensor(54.3907, grad_fn=<MseLossBackward0>) tensor([ 2.4272, -2.0785, -1.0354,  0.2732, -0.4750, -0.6053, -0.4538,  0.2985,
         1.4427, -0.3811,  0.2589, -0.5102,  1.0318, -0.9978, -0.8853, -1.0514,
        -0.2445,  0.1041, -0.4611,  0.6796, -0.2208, -0.9056,  0.7682,  0.4794,
         0.8367,  1.2635, -0.9853,  1.9676, -1.3036, -1.8072,  0.4253, -1.3689,
        -1.5129, -1.2250,  2.7457,  2.0146, -3.2404, -3.3948, -3.3887, -4.2334]) tensor([ 0.1985, -0.2981, -0.1796, -0.1065, -0.0989,  0.2372, -0.3311, -0.1812,
        -0.0091, -0.0843, -0.2636,  0.1832, -0.0435,  0.2756,  0.2815,  0.1610,
         0.0514, -0.1910, -0.0527,  0.1134, -0.1045, -0.0957,  0.2497, -0.1215,
         0.3243,  0.1376, -0.2712, -0.0478, -0.3024, -0.1488, -0.1151, -0.2414,
         0.0184, -0.1014,  0.2378,  0.1883, -0.3841,  0.2652, -0.3120, -0.4675],
       grad_fn=<CatBackward0>)
torch.Size([100, 1])
torch.Size([100, 1])
0 tensor(51.7729, grad_fn=<MseLossBac

torch.Size([100, 1])
torch.Size([100, 1])
4 tensor(40.1780, grad_fn=<MseLossBackward0>) tensor([-1.4082e-02,  2.3752e-03,  4.4207e-02, -7.2746e-02,  6.9480e-02,
         5.4749e-02,  7.1050e-02,  1.1311e-02, -7.1166e-03,  6.1126e-01,
        -2.5700e+00,  3.1392e+00, -4.3001e+00,  3.7666e+00,  3.9933e+00,
         3.9523e+00, -4.5018e+00,  7.4920e-01,  8.1520e-01, -1.2315e+00,
         9.5022e-01, -1.4725e+00,  1.4339e+00,  1.3457e+00,  1.4000e+00,
        -1.4240e+00,  1.4265e+00, -2.9005e-02,  1.3664e-01, -2.9442e-01,
         6.0074e-01, -6.0198e-01, -4.1437e-01, -6.4277e-01,  1.2436e-01,
         6.4112e-02, -2.3616e-01,  4.9733e+00, -2.5318e+00,  4.2292e-01]) tensor([ 0.0241, -0.1469, -0.0726, -0.1507, -0.0492,  0.2585, -0.2854, -0.1615,
        -0.0760,  0.1613, -0.2541,  0.1134, -0.7382,  0.8048,  0.7243,  0.7524,
         0.9633, -0.1949, -0.1352,  0.2126, -0.1820,  0.0024,  0.1533, -0.2218,
         0.2300,  0.1839, -0.3620, -0.1919, -0.1946, -0.0114, -0.1350, -0.1113,
       

8 tensor(15.3835, grad_fn=<MseLossBackward0>) tensor([-5.0743e-01,  4.8402e-01,  4.3793e-02, -3.9744e-01,  3.1748e-01,
         2.8579e-01,  3.4084e-01,  1.4540e-01, -5.2081e-02, -8.7417e-01,
        -1.5622e-01,  3.1678e-01, -7.4623e-01, -6.4213e-01,  9.6709e-01,
        -5.2387e-01, -2.0264e-01,  5.6988e-02,  1.6704e+00, -2.1128e+00,
         3.1381e-01, -5.5012e-01, -5.2287e-01, -8.7057e-03, -3.4717e-01,
         3.2350e-01,  6.6443e-01, -7.7435e-02,  5.5116e-02,  1.0860e-01,
         1.1300e-01,  7.8292e-04,  2.6731e-02, -1.5846e-02, -1.3508e-01,
        -2.2348e-02, -9.0272e-01, -3.7119e-01, -2.4165e+00,  5.9251e-02]) tensor([ 9.2720e-02, -2.0989e-01, -1.2520e-01, -9.5987e-02, -1.0191e-01,
         2.2755e-01, -3.3719e-01, -1.8946e-01, -5.8157e-02,  5.1002e-01,
        -2.2646e-01, -5.7307e-01, -5.5648e-01,  5.6892e-01,  4.1022e-01,
         5.5780e-01,  1.8173e+00,  9.0777e-02, -7.8823e-01,  9.6813e-01,
        -4.7348e-01,  3.9357e-01, -1.7070e-01, -6.0789e-01, -8.1995e-02,
    

         3.2812e-02, -1.2265e+00, -2.9006e-01, -1.0738e+00,  1.0753e-01]) tensor([ 0.3833, -0.4952, -0.0833,  0.0210, -0.2126,  0.1188, -0.4518, -0.2451,
         0.0219,  0.9180, -0.3705, -0.7941, -0.2326,  0.4247,  0.0957,  0.4613,
         1.8267,  0.2738, -1.2286,  1.5762, -0.4053,  0.3193,  0.1651, -0.5123,
         0.2594,  0.3055, -0.3299, -0.1821, -0.1895, -0.0173, -0.1686, -0.0973,
         0.0821,  0.0178,  0.0445,  0.0410,  0.6997,  2.2380,  2.1255, -0.1937],
       grad_fn=<CatBackward0>)
torch.Size([100, 1])
torch.Size([100, 1])
12 tensor(9.2133, grad_fn=<MseLossBackward0>) tensor([-7.3631e-01,  8.5390e-01, -6.6186e-01, -2.9760e-01,  3.5143e-01,
         6.4365e-01,  3.6727e-01, -1.1974e-01, -2.7718e-01, -1.2841e+00,
         4.9511e-01, -3.2175e-01, -1.2123e+00,  1.4086e+00,  2.2373e+00,
         1.4657e+00, -1.1855e+00, -8.2640e-01, -2.8291e-01, -2.1491e-01,
        -1.5256e+00,  1.5747e-01, -4.2101e-01, -2.7476e-01, -2.7238e-01,
        -1.9849e+00, -1.9351e-01, -1.7273

17 tensor(6.2656, grad_fn=<MseLossBackward0>) tensor([-0.3355,  0.0814, -0.2115, -0.0968, -0.1803,  0.0086, -0.1713,  0.5480,
        -0.1740,  0.2440, -1.4408,  0.7908,  0.0168, -0.5397, -0.2965, -0.5530,
         0.5205, -0.6584, -0.9746, -0.0954,  1.2355, -0.0150, -0.6579, -0.6724,
        -1.0773, -0.9638,  0.2936,  0.0097,  0.0965, -0.1550,  0.0707, -0.0406,
         0.0084, -0.0167,  0.0046,  0.0232, -0.3447,  0.3126, -0.1614,  0.0916]) tensor([ 5.7464e-01, -6.7803e-01,  1.1678e-01,  9.8237e-02, -2.6787e-01,
         2.9363e-02, -5.0679e-01, -3.6839e-01,  1.8122e-02,  1.1527e+00,
        -3.7075e-01, -5.4237e-01, -4.8036e-02,  3.8377e-01, -1.3784e-01,
         4.3532e-01,  1.7329e+00,  3.0633e-01, -1.1725e+00,  1.7715e+00,
        -3.8248e-01,  3.1936e-01,  3.2349e-01, -4.7395e-01,  4.6397e-01,
         4.2263e-01, -2.1106e-01, -1.8962e-01, -1.8604e-01, -3.5326e-02,
        -1.6383e-01, -1.1034e-01,  7.0647e-02,  1.4480e-03,  7.1290e-02,
         5.8803e-02,  1.0206e+00,  2.1722e

...

88 tensor(3.7350, grad_fn=<MseLossBackward0>) tensor([ 0.3008, -0.0601,  0.0756,  0.0088,  0.1366,  0.2008,  0.0846,  0.2969,
         0.3856,  1.0192, -0.6763,  0.0128, -0.7648,  1.0444,  1.1730,  0.9446,
         0.2085,  0.5044, -0.1083, -0.1026, -0.0912, -0.6707,  0.5879,  1.0810,
         0.5213, -0.0575,  0.3458,  0.0012, -0.0070, -0.0078,  0.0857, -0.0736,
        -0.1338, -0.0807, -0.0794, -0.0168, -0.0135,  0.7288, -0.2391,  0.0809]) tensor([ 8.1120e-01, -3.8011e-01,  1.0004e+00, -3.8316e-01, -5.3113e-01,
         1.9097e-01, -4.4962e-01, -3.4188e-01, -4.8444e-01,  2.0018e+00,
         1.0605e+00, -6.2206e-01,  2.8430e-01,  2.9058e-01, -5.0620e-01,
         8.5811e-01,  1.1824e+00,  2.2767e-01,  3.5392e-02,  2.5625e+00,
        -2.5919e-01,  5.0535e-01,  2.5121e-01, -3.7023e-01,  6.3394e-01,
         5.4725e-01, -2.1558e-01, -2.5322e-01, -1.7293e-01, -1.2886e-01,
        -5.9765e-02, -1.0594e-01,  6.8317e-02, -1.9905e-03,  2.8497e-01,
         6.3210e-02,  1.5891e+00,  2.7159e

torch.Size([100, 1])
torch.Size([100, 1])
91 tensor(4.1234, grad_fn=<MseLossBackward0>) tensor([ 0.0822, -0.2604, -0.5568,  0.7426, -0.1075, -0.1445, -0.3700, -0.2461,
        -0.0646, -0.9220,  0.7850, -0.7763,  0.3569, -0.1256, -0.1920, -0.1790,
         0.2273,  0.2376,  0.4784, -0.3632, -1.3040,  0.4531, -0.3184, -0.0064,
        -0.3006, -0.5362,  1.1840, -0.0116, -0.0594,  0.0267, -0.1469,  0.0976,
         0.0830,  0.0935, -0.0179, -0.0573, -0.2443, -0.0653, -0.4510, -0.0170]) tensor([ 7.8727e-01, -3.7695e-01,  1.0052e+00, -4.1712e-01, -5.4932e-01,
         2.0966e-01, -4.6018e-01, -3.2593e-01, -5.2874e-01,  2.0097e+00,
         1.0939e+00, -6.1884e-01,  2.9294e-01,  2.7136e-01, -5.2543e-01,
         8.5209e-01,  1.1602e+00,  2.2544e-01,  5.9531e-02,  2.5819e+00,
        -2.6382e-01,  4.9567e-01,  2.4108e-01, -3.7818e-01,  6.2126e-01,
         5.4499e-01, -2.1960e-01, -2.5163e-01, -1.7035e-01, -1.3216e-01,
        -5.8335e-02, -1.0432e-01,  7.6582e-02,  2.1441e-03,  2.9608e-01,


94 tensor(4.2584, grad_fn=<MseLossBackward0>) tensor([-0.3347,  0.6991,  0.3749,  0.3362, -0.1074,  0.2875,  0.1004, -0.0983,
        -0.2127,  0.3565, -0.4471, -0.0877,  0.6870, -1.0423,  0.2006, -0.8888,
         0.3493,  0.4434, -0.0849, -0.1207,  0.2721, -0.0785, -0.2775, -0.9786,
        -0.0063, -0.0294,  0.2323,  0.0208, -0.0659,  0.0744, -0.0371,  0.0397,
         0.1361,  0.0116, -0.0166, -0.0136, -0.0472, -0.0525, -0.0692,  0.0036]) tensor([ 0.7839, -0.3850,  0.9988, -0.4401, -0.5630,  0.2302, -0.4686, -0.3214,
        -0.5510,  2.0382,  1.0990, -0.6119,  0.2974,  0.2628, -0.5159,  0.8507,
         1.1362,  0.2260,  0.0691,  2.6068, -0.2761,  0.4923,  0.2288, -0.3879,
         0.6095,  0.5209, -0.1731, -0.2520, -0.1661, -0.1357, -0.0550, -0.1053,
         0.0807,  0.0031,  0.3037,  0.0535,  1.6274,  2.7304,  2.5653, -0.3592],
       grad_fn=<CatBackward0>)
torch.Size([100, 1])
torch.Size([100, 1])
94 tensor(4.4008, grad_fn=<MseLossBackward0>) tensor([ 0.4974, -0.3422, -0.5101

torch.Size([100, 1])
torch.Size([100, 1])
96 tensor(3.6779, grad_fn=<MseLossBackward0>) tensor([ 0.3392, -0.1942, -0.0269,  0.0458,  0.0528,  0.0990,  0.0104,  0.2958,
         0.2656,  1.0825, -0.7002, -0.0303, -0.7720,  1.0657,  1.1854,  0.9623,
         0.1519,  0.5457, -0.0325, -0.1608, -0.1485, -0.6697,  0.5777,  1.0833,
         0.5086, -0.1076,  0.3442, -0.0267,  0.0095,  0.0136,  0.0821, -0.0714,
        -0.1296, -0.0781, -0.0713, -0.0500,  0.0179,  0.7339, -0.3004,  0.0807]) tensor([ 0.7630, -0.3859,  0.9995, -0.4676, -0.5759,  0.2385, -0.4780, -0.3066,
        -0.5784,  2.0469,  1.1291, -0.6103,  0.2874,  0.2708, -0.5278,  0.8682,
         1.1315,  0.2361,  0.0941,  2.6085, -0.2736,  0.4641,  0.2422, -0.3654,
         0.6137,  0.5403, -0.2177, -0.2513, -0.1626, -0.1401, -0.0510, -0.1063,
         0.0847,  0.0047,  0.3093,  0.0512,  1.6402,  2.7534,  2.5623, -0.3630],
       grad_fn=<CatBackward0>)
torch.Size([100, 1])
torch.Size([100, 1])
96 tensor(4.0826, grad_fn=<MseLossBac

torch.Size([100, 1])
torch.Size([100, 1])
99 tensor(4.3083, grad_fn=<MseLossBackward0>) tensor([ 0.4469, -0.3238, -0.5470,  0.4420,  0.0901, -0.2234, -0.1898, -0.4004,
         0.2991, -0.0471, -0.2381, -0.7810,  0.4543,  0.4887,  0.2214,  0.1532,
        -0.9744, -0.1588, -0.9756,  0.8913,  0.0153,  0.8952, -0.5228, -0.8637,
        -0.6120,  0.2115,  1.8331,  0.1881, -0.1934, -0.0707, -0.0701,  0.0646,
        -0.0110,  0.0642, -0.0168, -0.1469, -0.2185, -0.2663,  0.8880,  0.0631]) tensor([ 0.7608, -0.4005,  0.9888, -0.4916, -0.5911,  0.2566, -0.4911, -0.3009,
        -0.5912,  2.0581,  1.1468, -0.6014,  0.2956,  0.2636, -0.5323,  0.8657,
         1.1030,  0.2272,  0.1039,  2.6343, -0.2865,  0.4686,  0.2295, -0.3768,
         0.5986,  0.5178, -0.1779, -0.2512, -0.1582, -0.1432, -0.0485, -0.1073,
         0.0884,  0.0058,  0.3183,  0.0498,  1.6579,  2.7533,  2.5722, -0.3697],
       grad_fn=<CatBackward0>)
torch.Size([100, 1])
torch.Size([100, 1])
99 tensor(3.5338, grad_fn=<MseLossBac

In [13]:
parameters_to_vector(model.parameters())

tensor([ 0.7507, -0.3908,  0.9968, -0.4981, -0.5987,  0.2649, -0.4940, -0.2958,
        -0.5991,  2.0674,  1.1485, -0.6043,  0.3032,  0.2484, -0.5323,  0.8558,
         1.1065,  0.2295,  0.0843,  2.6632, -0.2868,  0.4802,  0.2135, -0.4030,
         0.5820,  0.5223, -0.1598, -0.2478, -0.1611, -0.1418, -0.0495, -0.1052,
         0.0934,  0.0084,  0.3210,  0.0434,  1.6643,  2.7589,  2.5993, -0.3700],
       grad_fn=<CatBackward0>)

In [14]:
class CNN_Net(nn.Module):
    def __init__(self):
        super(CNN_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        output = self.fc2(x)
        return output

In [15]:
class ClientUpdate(object):
    def __init__(self, dataset, batchSize, alpha, lamda, epochs, projection_list, projected_weights):
        self.train_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=batchSize, shuffle=True)
        #self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchSize = batchSize

    def train(self, model):
        criterion = nn.MSELoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.5)

        e_loss = []
        for epoch in range(1, self.epochs+1):
            train_loss = 0
            model.train()
            for i, (data, labels) in zip(range(1), self.train_loader):
                data, labels = data, labels
                optimizer.zero_grad() 
                output = model(data)  
                loss = criterion(output, labels)
                #loss += mu/2 * torch.norm(client_param.data - server_param.data)**2
                loss.backward()
                grads = grads_to_vector(model.parameters())
                #optimizer.step()
                train_loss += loss.item()*data.size(0)
                weights = parameters_to_vector(model.parameters())
                mat_vec_sum = torch.zeros_like(weights)
                for j in G.neighbors(model.user_id):
                    mat_vec_sum = torch.add(mat_vec_sum, torch.matmul(torch.transpose(projection_list[model.user_id][j], 0, 1), 
                                                         projected_weights[j][model.user_id] - projected_weights[model.user_id][j]))
                
                model_update = parameters_to_vector(model.parameters()) - alpha * (grads + lamda * mat_vec_sum)
                
            vector_to_parameters(parameters=model.parameters(), vec=model_update)
                

            train_loss = train_loss/self.batchSize#len(self.train_loader.dataset) 
            e_loss.append(train_loss)

        total_loss = e_loss#sum(e_loss)/len(e_loss)

        return model.state_dict(), total_loss

In [16]:
# Preparing projection matrices
models = [MLP_Net(user_id=i) for i in range(no_users)]
#temp = MLP_Net()
projection_list = []
projected_weights = []

def update_ProjWeight(projection_list, projected_weights, first_run=True):
    #projected_weights = []
    for i in range(no_users):
        neighbors_mat = []
        neighbors_weights = []
        for j in range(no_users):
            if j in G.neighbors(i):
                with torch.no_grad():
                    if first_run == True:
                        row, column = d0, parameters_to_vector(models[i].parameters()).size()[0]
                        mat = torch.zeros((row, column))
                        mat.fill_diagonal_(1.0 + 1.0 * float(np.random.randn(1)))
                        neighbors_mat.append(mat)
                        neighbors_weights.append(torch.matmul(mat, parameters_to_vector(models[i].parameters())))
                    else:
                        neighbors_weights.append(torch.matmul(projection_list[i][j], parameters_to_vector(models[i].parameters())))
            else:
                neighbors_mat.append(0)
                neighbors_weights.append(0)
        if first_run == True:
            projection_list.append(neighbors_mat)
        projected_weights.append(neighbors_weights)

update_ProjWeight(projection_list, projected_weights)



In [17]:
print(projection_list[0])

[0, 0, tensor([[1.5772, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.5772, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.5772, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0

In [18]:
def testing(model, dataset, bs, criterion): 
    test_loss = 0
    correct = 0
    test_loader = DataLoader(MyDataset(X_test, y_test), batch_size=bs)
    l = len(test_loader)
    model.eval()
    for data, labels in test_loader:
        data, labels = data, labels
        output = model(data)
        loss = criterion(output, labels)
        test_loss += loss.item()*data.size(0)
        #_, pred = torch.max(output, 1)
        #correct += pred.eq(labels.data.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    
    return test_loss

In [19]:
def rel_error(model):
    return (torch.norm(parameters_to_vector(model.parameters()) - datapoints[model.user_id]['exact_weights']) / torch.norm(datapoints[model.user_id]['exact_weights'])).detach()

In [20]:
torch.norm(datapoints[model.user_id]['exact_weights'])

tensor(5.0990, dtype=torch.float64)

In [21]:
model = MLP_Net(user_id=0)

from torch.nn.utils import parameters_to_vector, vector_to_parameters

with torch.no_grad():    
    params = parameters_to_vector(model.parameters())

    print(params)

params *= 2.

vector_to_parameters(parameters=model.parameters(), vec=params)

parameters_to_vector(model.parameters())





tensor([ 2.5791e-01, -1.1576e-01, -1.8839e-01, -3.0653e-01,  3.2938e-02,
         1.2947e-01, -2.4702e-01, -2.6247e-01,  1.0280e-01, -3.4274e-02,
        -2.9882e-01,  1.1148e-01, -7.8249e-04,  3.1218e-01, -1.3248e-01,
        -1.0801e-01,  7.4604e-03,  2.9790e-01, -8.8858e-02, -1.7427e-01,
        -9.1826e-02, -6.6660e-02,  2.4504e-01, -2.0514e-01,  3.7661e-04,
         2.3968e-01,  5.0011e-02,  1.2925e-01,  6.7731e-03, -2.1153e-01,
         2.5250e-01, -2.7403e-01, -1.0404e-01, -1.5580e-01,  7.3808e-02,
         1.3833e-01, -6.9302e-03, -3.3627e-01,  1.4531e-01, -4.7420e-01])


tensor([ 5.1583e-01, -2.3153e-01, -3.7678e-01, -6.1306e-01,  6.5875e-02,
         2.5895e-01, -4.9404e-01, -5.2493e-01,  2.0559e-01, -6.8548e-02,
        -5.9763e-01,  2.2296e-01, -1.5650e-03,  6.2436e-01, -2.6496e-01,
        -2.1603e-01,  1.4921e-02,  5.9580e-01, -1.7772e-01, -3.4855e-01,
        -1.8365e-01, -1.3332e-01,  4.9007e-01, -4.1028e-01,  7.5322e-04,
         4.7937e-01,  1.0002e-01,  2.5850e-01,  1.3546e-02, -4.2307e-01,
         5.0500e-01, -5.4807e-01, -2.0809e-01, -3.1160e-01,  1.4762e-01,
         2.7666e-01, -1.3860e-02, -6.7254e-01,  2.9063e-01, -9.4841e-01],
       grad_fn=<CatBackward0>)

In [22]:
#global_model = CNN_Net().cuda()
models = [MLP_Net(user_id=i) for i in range(no_users)]
dummy_models = [MLP_Net(user_id=i) for i in range(no_users)]

#model.load_state_dict(global_model.state_dict())

criterion = nn.MSELoss()


train_loss = []
test_loss = []
test_accuracy = []
total_rel_error = []

for curr_round in tqdm(range(1, it+1)):
    w, local_loss = [], []

    
    for i in range(no_users):
        dummy_models[i].load_state_dict(models[i].state_dict())
        local_update = ClientUpdate(dataset=datapoints[i], batchSize=batch_size, alpha=alpha, lamda=lamda, epochs=1, projection_list=projection_list, projected_weights=projected_weights)
        weights, loss = local_update.train(dummy_models[i])
        w.append(weights)
        local_loss.append(loss)
        models[i].load_state_dict(w[i])
    
    if curr_round == 60:
        alpha *= 0.8
    if curr_round == 200:
        alpha *= 0.8
        
    
        
    
    
    # Update prjection matrix
    
    #print(projection_list[0], projected_weights[0])
    
    for i in range(no_users):
        weights = parameters_to_vector(models[i].parameters())
        for j in G.neighbors(i):
            mat_vec_sum = torch.zeros(d0)
            for k in G.neighbors(i):
                mat_vec_sum = torch.add(mat_vec_sum, projected_weights[i][k] - projected_weights[k][i])
            temp_mat = torch.outer(mat_vec_sum, weights).clone()


            projection_list[i][j] = torch.add(projection_list[i][j], -1 * eta * lamda * temp_mat)
                                         
    projected_weights = []                                          
    update_ProjWeight(projection_list, projected_weights, first_run=False)
        
        
        
    
    




          
            

    local_test_acc = []
    local_test_loss = []
    user_rel_error = 0
    for k in range(no_users):
      
        g_loss = testing(models[i], datapoints[i], 50, criterion)
        local_test_loss.append(g_loss)
        #user_rel_error += rel_error(models[i])
    
    
        

    g_loss = sum(local_test_loss) / len(local_test_loss)
    #total_rel_error.append(user_rel_error / no_users)
    
    

    test_loss.append(g_loss)
    #test_accuracy.append(g_accuracy)
    print("Training_loss %2.5f"% (test_loss[-1]))

  0%|          | 1/2000 [00:01<52:02,  1.56s/it]

Training_loss 48.21755


  0%|          | 2/2000 [00:02<43:37,  1.31s/it]

Training_loss 47.92515


  0%|          | 3/2000 [00:03<42:02,  1.26s/it]

...

Training_loss 24.65088


100%|█████████▉| 1999/2000 [32:18<00:01,  1.01s/it]

Training_loss 24.98538


100%|██████████| 2000/2000 [32:19<00:00,  1.03it/s]

Training_loss 25.19851


In [23]:
#Training_loss 5.33078 with no communication

In [24]:
#plot.plot(test_loss)
parameters_to_vector(models[19].parameters())

tensor([ 0.6498,  0.5638, -0.8521,  0.1079,  0.2085,  0.3570,  0.2455,  0.0381,
        -0.2764,  1.8908,  1.1659, -0.2855,  0.1584,  0.1103, -1.0718,  1.0543,
         0.9312,  0.5311, -0.0779,  2.0297, -0.3009,  0.7207,  0.5131, -0.3004,
         0.5323,  0.6547, -0.2666, -0.0088, -0.5496,  0.5120, -0.9995, -0.6847,
        -0.3101, -0.1495, -0.3391, -1.6521,  1.0532,  2.6998,  2.1916,  2.0958],
       grad_fn=<CatBackward0>)

In [25]:
for j in G.neighbors(0):
    print(j)

2
3
4
7
8


In [26]:
parameters_to_vector(models[0].parameters())

tensor([-0.1523,  0.0142, -0.7284, -0.3574,  0.1754,  0.0159, -0.6979, -0.7857,
        -0.3527, -1.6757, -1.3106,  0.5779, -0.4595,  0.7925, -0.6702,  0.0761,
         0.7484,  0.5895, -1.4977, -0.0773, -0.2616,  0.4856, -0.2582,  0.0864,
         0.1666,  0.7773, -0.5446, -0.6613, -2.6545,  0.2856, -0.1957,  0.3802,
        -0.3286,  0.2477,  0.4638, -0.4916,  1.2379,  2.4534,  1.6759,  2.6629],
       grad_fn=<CatBackward0>)

In [27]:
projection_list[0]

[0,
 0,
 tensor([[ 1.5768e+00, -1.1026e-03,  1.0094e-03,  1.4180e-04, -3.7897e-04,
           7.6718e-04,  1.4973e-03,  9.5909e-04,  7.7417e-04, -3.6380e-03,
           3.9821e-03,  1.2326e-03, -1.5220e-03, -2.0922e-03, -5.2375e-04,
          -2.9232e-03,  4.1438e-03,  9.2690e-04, -2.0173e-03,  3.4648e-03,
          -8.5586e-04,  7.5988e-05,  1.4284e-03,  1.8537e-03,  2.6821e-03,
          -2.1515e-04,  1.8757e-03,  7.8412e-03, -3.9963e-03,  1.2591e-03,
          -7.6591e-04, -7.5186e-04, -2.8237e-04,  2.0514e-03,  2.3287e-03,
          -1.6691e-03, -2.6062e-03,  5.3221e-03,  3.1952e-03,  6.5137e-03],
         [ 1.6101e-02,  1.5855e+00,  1.8589e-02,  1.7856e-02,  5.0975e-03,
          -2.2872e-03,  7.6694e-03,  2.3616e-02,  6.6366e-03,  5.6028e-02,
           1.3587e-02, -2.6815e-02,  6.6396e-03, -1.6604e-02,  2.2693e-02,
           8.7667e-03, -5.0490e-02, -1.7764e-02,  5.3661e-02, -5.1513e-03,
           2.0821e-02, -8.0512e-03, -2.0383e-03, -8.9978e-03, -1.7928e-02,
          -2.067

In [28]:
projected_weights[0]

[0,
 0,
 tensor([-0.1959, -1.2806, -0.6038, -0.4655,  0.6666, -0.3432, -0.3384, -0.0072,
         -1.3709]),
 tensor([-0.0094, -1.2979,  0.2881, -0.0279,  0.4518, -0.3627,  0.5162,  0.9549,
         -0.9390]),
 tensor([-0.1605, -1.2838, -0.4346, -0.3825,  0.6259, -0.3469, -0.1762,  0.1753,
         -1.2889]),
 0,
 0,
 tensor([ 0.0796, -1.3062,  0.7138,  0.1810,  0.3493, -0.3720,  0.9241,  1.4141,
         -0.7329]),
 tensor([-0.0649, -1.2927,  0.0229, -0.1580,  0.5157, -0.3569,  0.2621,  0.6688,
         -1.0674]),
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [29]:
test_loss = np.array(test_loss)
total_rel_error = np.array(total_rel_error)

In [30]:
print(test_loss)

[48.21755084 47.92515052 47.57178508 ... 24.65088367 24.9853778
 25.19851112]


In [31]:
np.save( 'training_loss_sheave_fml_alpha' + str(alpha).replace('.', '_') + "_eta_"+ str(eta).replace('.', '_') +  '_pout' + str(pout).replace('.', '_') + '+d0_' + str(d0), test_loss)
#np.save('relative_error_sheave_fml' + str(lamda).replace('.', '_'), total_rel_error)

In [32]:
'training_loss_sheave_fml' + str(lamda).replace('.', '_'), test_loss

('training_loss_sheave_fml0_001',
 array([48.21755084, 47.92515052, 47.57178508, ..., 24.65088367,
        24.9853778 , 25.19851112]))